#  HEP-Th Performance experiments

In this notebook we measure if the Hep_th experiment is capable of optimizing its loss function.

In [ ]:
from tqdm import tqdm

In [ ]:
import time
import numpy as np
import torch
import sys
import pandas as pd

sys.path.append('../pytorch_graph_edit_networks')
sys.path.append('../hep_th')

import pytorch_graph_edit_networks as gen
import hep_th

# model parameters
num_layers = 3
dim_hid = 64
nonlin = torch.nn.Tanh()

# initialize model, one without edge filters, one with edge filters, but no limit, and one
# with limit
filter_options = [True, 538]
# set up model names for result storage
model_names = ['flex_filter', 'const_filter']
comp_names  = ['forward', 'backward']
headers = ['sizes']
for model_name in model_names:
    for comp_name in comp_names:
        headers.append('%s_%s' % (model_name, comp_name))

max_past = 12


# set up a list of all experimental settings
month_tuples = []
for year in range(1992,2003+1):
    max_month = 12
    if year == 2003:
        max_month = 4
    for month in range(1, max_month+1):
        month_tuples.append((year, month))
settings = []
for past in range(1, max_past+1):
    for t in range(past, len(month_tuples)):
        settings.append((month_tuples[t][0], month_tuples[t][1], past))


In [ ]:
learning_rate  = 1E-3
weight_decay   = 1E-5

fee = 538

xx = 0
curves = {}
learning_curve = []

month_tuples = []
for year in range(1992,2003+1):
    max_month = 12
    if year == 2003:
        max_month = 4
    for month in range(1, max_month+1):
        month_tuples.append((year, month))
settings = []
for past in range(1, max_past+1):
    for t in range(past, len(month_tuples)):
        settings.append((month_tuples[t][0], month_tuples[t][1], past))

net = gen.GEN(num_layers = num_layers, dim_in = 1, dim_hid = dim_hid, nonlin = nonlin, filter_edge_edits = fee)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
learning_curve2 = []
for epoch in range(1):
    print(f'epoch: {epoch}')
    for iter, sets in tqdm(enumerate(settings)):
        optimizer.zero_grad()
        year, month, past = settings[iter]
        loss = hep_th.compute_loss(net, year, month, past = past)
        loss.backward()
        optimizer.step()
        learning_curve2.append(loss.item())
        (epoch*len(sets) + iter)
        if (epoch*len(sets) + iter)%100 == 0: print(loss.item())



In [ ]:
breakpoints = []
tau = 0
for ind, i in enumerate(settings):
    temp_tau = i[2]
    if temp_tau != tau:
        tau = temp_tau
        breakpoints.append(ind)

print(breakpoints)
import seaborn as sns
from matplotlib import pyplot as plt
#sns.lineplot(x=range(len(learning_curve2)), y=learning_curve2)

tau_3 = learning_curve2[breakpoints[2]:breakpoints[3]]
tau_7 = learning_curve2[breakpoints[6]:breakpoints[7]]
tau_12 = learning_curve2[breakpoints[11]:]


taus = {'tau_3' : tau_3, 'tau_7' : tau_7, 'tau_12' : tau_12}


In [ ]:
df = pd.DataFrame.from_dict(taus, orient='index').T
df = df.iloc[:123, :]
df

In [ ]:
melted = pd.melt(df.reset_index(), id_vars=['index'])
melted.columns = ['iteration', r'$\tau$', 'loss']
melted

In [ ]:
import seaborn as sns
plt.style.use('seaborn')
plt.rcParams['text.usetex'] = True
#sns.lineplot(data=melted, x='id', y='val', hue='var')
#plt.show()
fig, ax = plt.subplots(figsize=(6,3))
sns.lineplot(data=melted, x='iteration', y='loss', hue=r'$\tau$', ax=ax)
plt.savefig('LossCurves.png', bbox_inches='tight')
plt.show()

In [ ]:
### Convert the files to be compatible with the visualization script in R

import pandas as pd
d = np.loadtxt(f'hep_th_runtimes_1.csv', delimiter='\t', skiprows = 1)
df = pd.DataFrame(d)
df = df[range(5)]
df.columns = ['Graph size', 'Flexible forward', 'Constant forward', 'Flexible backward', 'Constant backward']
df.to_csv(f'hep_th_runtimes_conv_1.csv', index=False)